Julia Veloso Dias

Projeto Integrador 2025

# Instalar dependências

In [1]:

!pip install langchain langchain-community faiss-cpu sentence-transformers transformers torch accelerate pandas

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# Imports

In [2]:
# bibliotecas padrão
import os
import json
import re
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

# nlp
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# transformers e modelos
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from sentence_transformers import SentenceTransformer, util

# Estrutura de dados

Esta etapa do DocumentAnalyzer tem a funcionalidade de realizar uma análise estrutural e conversão dos documentos institucionais para o formato adequado ao sistema RAG. A classe atua como um "preparador de documentos" que examina a qualidade, organização e consistência dos dados de entrada, identificando campos obrigatórios, contando artigos e parágrafos, detectando inconsistências e calculando um score de qualidade.

Em seguida, converte essa estrutura em documentos LangChain padronizados, aplicando estratégias diferentes para páginas, artigos e tabelas, enquanto enriquece os metadados para melhor recuperação de informação.

Essencialmente, transforma dados brutos em estruturado e otimizado para o pipeline de perguntas e respostas.

In [3]:
class DocumentAnalyzer:
    def __init__(self):
        self.analysis_results = {}

    def analyze_document_structure(self, data: Dict, doc_name: str) -> Dict:
        """Analisa a estrutura do documento"""
        analysis = {
            'document_name': doc_name,
            'has_required_fields': False,
            'missing_fields': [],
            'structure_type': 'unknown',
            'articles_count': 0,
            'paragraphs_count': 0,
            'tables_count': 0,
            'pages_count': 0,
            'inconsistencies': [],
            'recommendations': []
        }

        # campos esperados
        required_fields = ['doc_id', 'nome_doc', 'estrutura']
        optional_fields = ['versao', 'data_publicacao', 'pagina_inicial', 'pagina_final', 'tables', 'paginas']

        # campos obrigatórios
        missing_required = [field for field in required_fields if field not in data]
        analysis['missing_fields'] = missing_required
        analysis['has_required_fields'] = len(missing_required) == 0

        # identificar estrutura
        if data.get('paginas') and len(data['paginas']) > 0:
            analysis['structure_type'] = 'paginas'
            analysis['pages_count'] = len(data['paginas'])
        elif data.get('estrutura') and len(data['estrutura']) > 0:
            analysis['structure_type'] = 'estrutura'
        elif data.get('tables') and len(data['tables']) > 0:
            analysis['structure_type'] = 'tables'
        else:
            analysis['structure_type'] = 'minimal'

        # conta artigos e paragrafos
        if analysis['structure_type'] == 'estrutura':
            for article in data['estrutura']:
                analysis['articles_count'] += 1
                paragraphs = article.get('paragrafos', [])
                analysis['paragraphs_count'] += len(paragraphs)

                if not article.get('artigo'):
                    analysis['inconsistencies'].append(f"Artigo sem identificador na posição {analysis['articles_count']}")

                # verificar parágrafos vazios
                for i, paragraph in enumerate(paragraphs):
                    if not paragraph.get('texto') or len(paragraph.get('texto', '').strip()) < 10:
                        analysis['inconsistencies'].append(f"Parágrafo vazio: Artigo {article.get('artigo', '?')} - Parágrafo {i+1}")

        # contar tabelas
        if data.get('tables'):
            analysis['tables_count'] = len(data['tables'])
            for i, table in enumerate(data['tables']):
                if not table or len(table) == 0:
                    analysis['inconsistencies'].append(f"Tabela {i+1} vazia")

        return analysis

    def convert_to_langchain_documents(self, data: Dict, doc_name: str) -> List[Document]:
        """Converte para estrutura LangChain com metadados ricos"""
        documents = []

        # conteúdo por páginas
        if data.get('paginas'):
            for page in data['paginas']:
                content = self._extract_page_content(page)
                if content:
                    metadata = {
                        'doc_id': data.get('doc_id', 'unknown'),
                        'doc_name': doc_name,
                        'page_number': page.get('page', 'unknown'),
                        'chapter': page.get('chapter', ''),
                        'article': page.get('article', ''),
                        'content_type': 'page',
                        'source': f"{doc_name} - Página {page.get('page', '?')}"
                    }
                    documents.append(Document(page_content=content, metadata=metadata))

        # conteúdo estruturado
        elif data.get('estrutura'):
            for article in data['estrutura']:
                content = self._extract_article_content(article)
                if content:
                    metadata = {
                        'doc_id': data.get('doc_id', 'unknown'),
                        'doc_name': doc_name,
                        'article': article.get('artigo', ''),
                        'chapter': article.get('capitulo', ''),
                        'section': article.get('secao', ''),
                        'content_type': 'article',
                        'source': f"{doc_name} - {article.get('artigo', 'Artigo')}"
                    }
                    documents.append(Document(page_content=content, metadata=metadata))

        # tabelas
        if data.get('tables'):
            for i, table in enumerate(data['tables']):
                table_content = self._extract_table_content(table)
                if table_content:
                    metadata = {
                        'doc_id': data.get('doc_id', 'unknown'),
                        'doc_name': doc_name,
                        'table_index': i,
                        'content_type': 'table',
                        'source': f"{doc_name} - Tabela {i+1}"
                    }
                    documents.append(Document(page_content=table_content, metadata=metadata))

        return documents

    def _extract_page_content(self, page: Dict) -> str:
        """Extrai conteúdo de uma página"""
        content_parts = []

        # prioridade: texto limpo → texto bruto
        if page.get('clean_text'):
            content_parts.append(page['clean_text'])
        elif page.get('raw_text'):
            content_parts.append(page['raw_text'])

        # metadados estruturais
        if page.get('chapter'):
            content_parts.append(f"[Capítulo: {page['chapter']}]")
        if page.get('article'):
            content_parts.append(f"[Artigo: {page['article']}]")

        return " ".join(content_parts) if content_parts else ""

    def _extract_article_content(self, article: Dict) -> str:
        """Extrai conteúdo de um artigo"""
        content_parts = []

        # cabeçalho do artigo
        if article.get('artigo'):
            content_parts.append(article['artigo'])

        if article.get('capitulo'):
            content_parts.append(f"Capítulo: {article['capitulo']}")

        if article.get('secao'):
            content_parts.append(f"Seção: {article['secao']}")

        # parágrafos
        for paragraph in article.get('paragrafos', []):
            if paragraph.get('texto') and len(paragraph['texto'].strip()) > 5:
                para_text = paragraph['texto']
                if paragraph.get('numero'):
                    para_text = f"{paragraph['numero']} {para_text}"
                content_parts.append(para_text)

        return "".join(content_parts) if content_parts else ""

    def _extract_table_content(self, table: List) -> str:
        """Extrai conteúdo de tabela de forma inteligente"""
        if not table or not isinstance(table, list):
            return ""

        content_lines = []

        for row in table:
            if isinstance(row, list):
                # filtrar células vazias e juntar
                cells = [str(cell).strip() for cell in row if cell and str(cell).strip()]
                if cells and not self._is_institutional_row(cells):
                    content_lines.append(" | ".join(cells))
            elif isinstance(row, str) and row.strip():
                if not self._is_institutional_row([row]):
                    content_lines.append(row.strip())

        return " ".join(content_lines) if content_lines else ""

    def _is_institutional_row(self, cells: List[str]) -> bool:
        """Identifica linhas institucionais irrelevantes"""
        text = " ".join(cells).upper()

        institutional_indicators = [
            'INSTITUTO FEDERAL', 'CAMPUS', 'DIRETORIA', 'MINISTÉRIO',
            'SECRETARIA', 'COORDENAÇÃO', 'REITORIA', 'FORMULÁRIO'
        ]

        return any(indicator in text for indicator in institutional_indicators)

    def generate_analysis_report(self, analyses: Dict) -> pd.DataFrame:
        """Gera relatório consolidado da análise"""
        report_data = []

        for doc_name, analysis in analyses.items():
            report_data.append({
                'Documento': doc_name,
                'Tipo Estrutura': analysis['structure_type'],
                'Campos Obrigatórios': 'OK' if analysis['has_required_fields'] else 'Erro',
                'Artigos': analysis['articles_count'],
                'Parágrafos': analysis['paragraphs_count'],
                'Tabelas': analysis['tables_count'],
                'Inconsistências': len(analysis['inconsistencies'])
            })

        return pd.DataFrame(report_data)

# carregar e analisar documentos
def load_and_analyze_documents(json_directory: str) -> Tuple[Dict, List[Document]]:
    """Carrega e analisa todos os documentos JSON"""
    analyzer = DocumentAnalyzer()
    all_documents = []
    analyses = {}

    json_files = list(Path(json_directory).glob('*.jsonl'))

    print(f"Encontrados {len(json_files)} arquivos JSON")

    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            doc_name = json_file.stem

            # Análise da estrutura
            analysis = analyzer.analyze_document_structure(data, doc_name)
            analyses[doc_name] = analysis

            # Conversão para documentos LangChain
            documents = analyzer.convert_to_langchain_documents(data, doc_name)
            all_documents.extend(documents)

            print(f"{len(documents)} documentos convertidos")

        except Exception as e:
            print(f"Erro ao processar {json_file}: {e}")

    return analyses, all_documents

json_directory = '/content/sample_data/'
analyses, documents = load_and_analyze_documents(json_directory)

# relatório
if analyses:
    report_df = DocumentAnalyzer().generate_analysis_report(analyses)
    print(f"\nTotal de documentos LangChain: {len(documents)}")
else:
    print("Nenhum documento foi analisado com sucesso")

Encontrados 2 arquivos JSON
37 documentos convertidos
322 documentos convertidos

Total de documentos LangChain: 359


# Embedding e vetorização

A classe VectorStoreManager funciona como o parte semântica do sistema RAG, responsável por transformar os documentos textuais em representações numéricas (embeddings) e criar um mecanismo de busca que compreende o significado por trás das palavras.

Utilizando o modelo `all-MiniLM-L6-v2` para gerar embeddings de qualidade e a biblioteca FAISS para indexação vetorial, ela converte todo o conteúdo em um espaço semântico onde documentos com significados similares ficam próximos, permitindo que o sistema recupere informações relevantes mesmo quando as palavras exatas da consulta não estão presentes nos textos originais.

In [4]:
class VectorManager:
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.embeddings = None
        self.vectorstore = None

    def create_embeddings(self):
        """Cria os embeddings usando sentence-transformers"""
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.model_name,
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        print("Embeddings configurado")

    def create_vector_store(self, documents: List[Document]):
        """Cria o vector store com FAISS"""
        if not documents:
            raise ValueError("Nenhum documento fornecido")

        self.vectorstore = FAISS.from_documents(documents, self.embeddings)
        print(f"Vector store criado: {len(documents)}")

        # Estatísticas do vector store
        index = self.vectorstore.index
        print(f"Dimensões dos embeddings: {index.d}")
        print(f"Total de vetores no índice: {index.ntotal}")

    def search_similarity(self, query: str, k: int = 5) -> List[Document]:
        """Busca por similaridade semântica"""
        if not self.vectorstore:
            raise ValueError("Vector store não inicializado")

        return self.vectorstore.similarity_search(query, k=k)

    def get_retriever(self, search_type: str = "similarity", k: int = 4):
        """Retorna um retriever configurado"""
        search_kwargs = {"k": k}

        if search_type == "mmr":
            search_kwargs.update({"fetch_k": 10, "lambda_mult": 0.7})

        return self.vectorstore.as_retriever(
            search_type=search_type,
            search_kwargs=search_kwargs
        )

vector_manager = VectorManager()

# criar embeddings
vector_manager.create_embeddings()

# criar vector store
vector_manager.create_vector_store(documents)

# testar busca
test_query = "tempo máximo conclusão curso"
similar_docs = vector_manager.search_similarity(test_query, k=3)
print(f"\nTeste de busca: '{test_query}'")
print(f"Documentos encontrados: {len(similar_docs)}")

for i, doc in enumerate(similar_docs):
    print(f"   {i+1}. {doc.metadata['source']}")
    print(f"      {doc.page_content[:100]}...")

/tmp/ipython-input-3128124425.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings configurado
Vector store criado: 359
Dimensões dos embeddings: 384
Total de vetores no índice: 359

Teste de busca: 'tempo máximo conclusão curso'
Documentos encontrados: 3
   1. ppc_output - Tabela 202
      2 IDENTIFICAÇÃO DO CURSO Denominação do Curso: Ciência da Computação Carga horária total: 3.280 hora...
   2. estagio_output - Artigo
      DO INÍCIO E DURAÇÃO...
   3. ppc_output - Tabela 300
      6.5.13 Trabalho de conclusão de curso O Trabalho de Conclusão de Curso (TCC) é um requisito curricul...


# Modelos LLM

Esta função tem como objetivo configurar e inicializar o modelo de linguagem DialoGPT-medium para ser utilizado no sistema de perguntas e respostas.

Ela carrega o tokenizer e o modelo pré-treinado da Microsoft, realiza configurações essenciais como a definição do token de padding (crítico para evitar erros de processamento) e cria um pipeline de geração de texto com parâmetros otimizados para conversação - incluindo controle de criatividade (temperature), diversidade lexical (top_p), prevenção de repetições (repetition_penalty) e limites de comprimento para garantir respostas coerentes e contextualizadas.

A função encapsula toda essa complexidade em uma interface simples que retorna um objeto LLM pronto para ser integrado ao pipeline RAG, proporcionando ao sistema a capacidade de gerar respostas conversacionais naturais baseadas nos documentos recuperados.

In [14]:
def setup_dialogpt_model():
    """Configura o modelo DialoGPT"""
    print("--- DIALOGPT ---")

    try:
        from langchain.llms import HuggingFacePipeline
        from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

        model_name = "microsoft/DialoGPT-large"

        tokenizer = AutoTokenizer.from_pretrained(model_name)

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(model_name)

        # configurar pipeline de geração com parâmetros
        text_generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=300,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            truncation=True,
            max_length=2048
        )

        llm = HuggingFacePipeline(pipeline=text_generator)
        print("OK")
        return llm

    except Exception as e:
        print(f"Erro {e}")


# RAG

Esta função implementa o núcleo do sistema RAG (Retrieval-Augmented Generation) conectando o mecanismo de recuperção de informações com o modelo de linguagem para gerar respostas contextualizadas.

Ela cria um pipeline onde, para cada pergunta, o sistema primeiro recupera os documentos mais relevantes do banco vetorial (usando similaridade semântica e buscando os 5 melhores resultados) e depois injeta esse contexto como base para o modelo DialoGPT gerar uma resposta precisa e fundamentada.

A função emprega uma abordagem de "stuffing" onde todo o contexto recuperado é inserido diretamente no prompt do modelo, seguido pela pergunta do usuário, criando uma estrutura limpa que permite ao LLM acessar as informações relevantes dos documentos institucionais.

In [15]:
def setup_rag_pipeline(vector_store_manager, llm):
    """Configura o pipeline RAG completo com tratamento de erro"""
    print("Configurando pipeline RAG...")

    from langchain.chains import RetrievalQA
    from langchain.prompts import PromptTemplate


    prompt_template = """

{context}

Pergunta: {question}

"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    try:
        # Configurar retriever com menos documentos para evitar sobrecarga
        retriever = vector_store_manager.get_retriever(
            search_type="similarity",
            k=5
        )

        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={"prompt": PROMPT},
            return_source_documents=True
        )


        return qa_chain

    except Exception as e:
        print(f"Erro {e}")
        return setup_simple_qa_chain(llm)

def setup_simple_qa_chain(llm):
    """Configura uma cadeia QA simples como fallback"""
    from langchain.chains import LLMChain
    from langchain.prompts import PromptTemplate

    prompt = PromptTemplate(
        input_variables=["question"],
        template="Responda à pergunta: {question}"
    )

    class SimpleQAChain:
        def __init__(self, chain):
            self.chain = chain

        def __call__(self, inputs):
            result = self.chain.run(inputs["query"])
            return {
                "result": result,
                "source_documents": []
            }

    chain = LLMChain(llm=llm, prompt=prompt)
    return SimpleQAChain(chain)

# PERGUNTA

Esta função implementa a interface principal de consulta ao sistema RAG, funcionando como o ponto de interação entre o usuário e a inteligência artificial.

Ela recebe perguntas, as submete ao pipeline RAG configurado e retorna respostas contextualizadas com transparência sobre as fontes consultadas. A função incorpora um sofisticado mecanismo de timeout (190 segundos) que monitora o tempo de processamento para evitar travamentos, garantindo responsividade mesmo com consultas complexas.

Após processar cada pergunta, ela exibe de forma organizada tanto a resposta gerada pelo modelo quanto a lista detalhada dos documentos institucionais que fundamentaram a resposta, incluindo previews do conteúdo recuperado.

In [ ]:
def ask_rag_question(qa_chain, question: str, question_num: int = None):
    """Faz uma pergunta ao pipeline RAG"""

    if question_num:
        print(f"\n{'='*60}")
        print(f"PERGUNTA {question_num}: {question}")
        print(f"{'='*60}")


    try:
        import signal
        import time

        def timeout_handler(signum, frame):
            raise TimeoutError("Tempo limite excedido")

        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(190)

        try:
            result = qa_chain({"query": question})
            signal.alarm(0)
        except TimeoutError:
            signal.alarm(0)
            raise TimeoutError("Processamento demorou muito tempo")


        cleaned_result = result['result'].strip()  # remove espaços e \n do início/fim
        if cleaned_result.startswith('\n'):
            cleaned_result = cleaned_result[1:]  # remove primeiro \n se existir


        print(f"\n**RESPOSTA:** {result['result']}")


        if result.get('source_documents'):
            print(f"\nFONTES CONSULTADAS ({len(result['source_documents'])}):")
            for i, doc in enumerate(result['source_documents'], 1):
                source_type = doc.metadata.get('content_type', 'documento').upper()
                print(f"   {i}. [{source_type}] {doc.metadata['source']}")


                preview = doc.page_content[:80] + "..." if len(doc.page_content) > 80 else doc.page_content
                print(f"      {preview}")
        else:
            print(f"\nAVISO: Nenhuma fonte foi recuperada para esta pergunta")

        return {
            "result": cleaned_result,  # Retorne a versão limpa
            "source_documents": result.get('source_documents', [])
        }

    except TimeoutError as e:
        print(f"ERRO DE TIMEOUT: {e}")
        return {
            "result": "Desculpe, a pergunta demorou muito para processar. Tente reformular ou perguntar algo mais específico.",
            "source_documents": []
        }
    except Exception as e:
        print(f"ERRO:{e}")
        return {
            "result": f"Erro ao processar a pergunta: {str(e)}",
            "source_documents": []
        }


def run_rag():
    """Executa pipeline RAG"""


    try:
        # llm

        llm = setup_dialogpt_model()

        # rag

        qa_chain = setup_rag_pipeline(vector_manager, llm)

        # perguntas
        test_questions = [
            "Quais são os professores que podem ser orientadores do TCC?",
            "Qual é o tempo máximo permitido para que um estudante conclua o curso",
            "Como o aluno sabe se foi aprovado ou reprovado numa matéria?",
            "Em qual momento do curso a gente pode começar a fazer o estágio?",
            "Quantas horas e que tipo de atividades contam como Atividades Complementares no curso deCiência da Computação?"
        ]

        results = []
        for i, question in enumerate(test_questions, 1):
            result = ask_rag_question(qa_chain, question, i)
            results.append({
                "question": question,
                "answer": result["result"],
                "sources": [doc.metadata for doc in result.get("source_documents", [])]
            })
            # espaço
            import time
            time.sleep(2)


        print(f"\nSALVANDO RESULTADOS")
        with open('rag_results.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

        print("Resultados salvos em 'rag_results.json'")


        return results

    except Exception as e:
        print(f"ERRO {e}")
        return []

# EXECUTAR TUDO
if __name__ == "__main__":
    results = run_rag()


CONFIGURANDO MODELO...
--- DIALOGPT ---


Device set to use cpu


OK

CONFIGURANDO PIPELINE RAG...
Configurando pipeline RAG...

EXECUTANDO 5 PERGUNTAS...

PERGUNTA 1: Quais são os professores que podem ser orientadores do TCC?

**RESPOSTA:** 

de TCC. E
m resumo, compete ao aluno após a realização da disciplina de PTCC:   Cumprir as etapas de trabalho estabelecidas no cronograma do projeto.   Cumprir o calendário estabelecido pelo professor orientador.   Comparecer aos horários de orientação definidos pelo professor orientador.   Realizar as atividades previstas no projeto.   Elaborar monografia redigida de acordo com as Normas Institucionais e com o modelo apre- sentado na disciplina de PTCC.   Entregar uma cópia do trabalho a cada membro da banca de avaliação, obedecendo aos pra- zos estipulados pelo calendário acadêmico e pelo professor orientador.   Apresentar oralmente o TCC perante banca de professores para avaliação.   Analisar juntamente com o professor orientador as sugestões da banca e efetuar as correções e adequações julgadas per

Na primeira consulta sobre professores orientadores, o modelo devolveu informações sobre responsabilidades dos alunos em TCC; na segunda pergunta sobre tempo máximo de conclusão do curso, retornou detalhes sobre carga horária do TCC; e na quarta questão sobre estágio, apresentou trechos burocráticos de avaliação em vez do momento de início; na quinta retornou a quantidade necessária para horas de atividade complementar e quais atividades contam como atividade complementar. Essas inconsistências revelam que, embora o mecanismo de recuperção esteja encontrando textos com similaridade lexical, falta precisão na seleção contextual. Todos os trechos vieram do documento, de forma bem similar ou igual. Torna-se importante, portanto, chegar melhor os parametros do modelo LLM. 

LinkCOLAB: https://colab.research.google.com/drive/1x7KN_gsf_fNPKcQczMA6pILGBJag5sSm?usp=sharing